# AI Cyoda configurations Q&A with RAG Langchain

Welcome to this Jupyter notebook! This notebook serves as your guide to developing an AI-powered Question & Answer system using the Langchain library. This system utilizes the Retrieval-Augmented Generation (RAG) model that leverages OpenAI's GPT-3 model to provide intelligent and context-aware responses.

The primary purpose of this notebook is to generate Cyoda related code. It does so by interacting with the data set available in the official Cyoda repository. 

## What will we cover?

In this notebook, we will go through the following steps:

1. **Setting up the environment**: We will install necessary libraries and load environment variables.

2. **Initializing the AI model**: We will initialize the ChatOpenAI model with the appropriate parameters.

3. **Loading instructions and entities**: We will load instructions and entities from the official repository using the GitLoader.

4. **Splitting documents and creating a vectorstore**: We will split the loaded documents into chunks and create a vectorstore using the Chroma library.

5. **Defining prompts for contextualizing and answering questions**: We will define prompts that the AI model will use to contextualize and answer questions.

6. **Creating a retrieval chain**: We will create a retrieval chain that combines the history-aware retriever and the question-answer chain.

7. **Running the chatbot**: Finally, we will run the chatbot and see it in action!

## Let's get started!

Please follow along with the code cells and comments to understand each step of the process. If you have any questions or run into any issues, feel free to ask for help. Happy coding!

Install requirements

In [1]:
%%script echo skipping
pip install -r ../requirements.txt

skipping


### Load environment variables

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
WORK_DIR = os.environ["WORK_DIR"]

In [3]:
%%script echo skipping
##for google colab (optional)
# This cell is optional and can be skipped
from google.colab import userdata
API_KEY = userdata.get('OPENAI_API_KEY')
WORK_DIR = userdata.get('WORK_DIR')

skipping


### Handle unsupported version of sqlite3 (optional)

In [4]:
%%script echo skipping
pip install pysqlite3-binary

skipping


In [5]:
%%script echo skipping
import sys

__import__("pysqlite3")
sys.modules["sqlite3"] = sys.modules["pysqlite3"]

skipping


### Initialize ChatOpenAI

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import GitLoader
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema import HumanMessage

In [7]:
llm = ChatOpenAI(
    temperature=0.7,
    max_tokens=8000,
    model="gpt-3.5-turbo-16k",
    openai_api_key=OPENAI_API_KEY,
)

### Load instructions and entities from the official cyoda repository

In [ ]:
loader = GitLoader(
    repo_path=WORK_DIR,
    branch="cyoda-ai-configurations-3.0.x",
    file_filter=lambda file_path: file_path.startswith(f"{WORK_DIR}/data/code/"),
)
docs = loader.load()
print(f"Number of documents loaded: {len(docs)}")

### Split documents and create vectorstore

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [10]:
count = vectorstore._collection.count()
print(count)

22


In [11]:
res = vectorstore.similarity_search("Get FOREIGN TRAVEL FOR LEEDS CITY COUNCIL")
print(res)

[Document(page_content='FOREIGN TRAVEL FOR LEEDS CITY COUNCIL (April 18 to March 19)     ALL COSTS IN GBP\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\nNAME OF PERSON TRAVELING\tJOB TITLE\tCITY\tCOUNTRY\tDEPARTURE DATE\tRETURN DATE\tTOTAL NUMBER OF DAYS AND NIGHTS\tREASON FOR VISIT \tCLASS OF FLIGHT i.e business\tTRAVEL COSTS                                                                                                                      \t\tCOST OF ACCOMMODATION \t\tEXPENSES - LOCAL TRANSPORT \t\tEXPENSES - FOOD & DRINK\t\tTOTAL COST OF VISIT\tTOTAL COSTS TO LCC\tTOTAL COSTS TO OTHER ORGANISATIONS (IF KNOWN )\r\n\t\t\t\t\t\t\t\t\tTo LCC\tTo other\tTo LCC\tTo other\tTo LCC\tTo other\tTo LCC\tTo other\t\t\t\r\nCllr Adam Ogilvie\tCouncillor\tBerlin\tGermany\t02.04.18\t04.04.18\t2 nights/3 days\tTo host a reception promoting VisitLeeds & Leeds2023\teconomy\t205.28\t0\t159.77\t0\t0\t0\t22.64\t0\t387.69\t387.69\t0', metadata={'file_name': '2018-2019-international-travel-expenses-csv-1.txt', '

### Define prompts for contextualizing question and answering question

In [13]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [14]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer question

In [15]:
qa_system_prompt = """You are a code assistant. You are aware of grpc client CyodaCalculationMemberClient. \
CalculationMemberGreetEvent.json, CalculationMemberJoinEvent.json, CloudEventType.json, EntityProcessorCalculationRequest.json, EntityProcessorCalculationResponse.json are used for schema generation. \
Also use available proto files.You should write code in different languages to produce similar grpc clients.. \
You should do your best to answer the question. \
Use the following pieces of retrieved context to answer the question. \

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

### Create retrieval chain

In [16]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Initialize chat history and relevant functions

In [17]:
chat_history = {}

In [18]:
# Function to add a message to the chat history
def add_to_chat_history(id, question, message):
    if id in chat_history:
        chat_history[id].extend([HumanMessage(content=question), message])
    else:
        chat_history[id] = [HumanMessage(content=question), message]

In [19]:
# Function to clear chat history
def clear_chat_history(id):
    if id in chat_history:
        del chat_history[id]

In [20]:
def ask_question(id, question):
    ai_msg = rag_chain.invoke(
        {"input": question, "chat_history": chat_history.get(id, [])}
    )
    add_to_chat_history(id, question, ai_msg["answer"])
    return ai_msg["answer"]

### Start a chat session

In [21]:
import uuid

# Generate a unique ID for the chat session
id = uuid.uuid1()

In [26]:
##Rewrite CyodaCalculationMemberClient to java. Produce ready to use code. Leave spring boot
question = "Write some logic java code in a @Component that does some processing of this entity. This should be a full code, use as much tokens as u need"
result = ask_question(id, question)
print(result)

Certainly! Here's a full Java code example of a `ForeignTravelProcessor` component that performs processing of the `ForeignTravel` entity:

```java
import org.springframework.stereotype.Component;

@Component
public class ForeignTravelProcessor {

    public void processForeignTravel(ForeignTravel foreignTravel) {
        calculateTotalCosts(foreignTravel);
        generateExpenseReport(foreignTravel);
        // Other processing tasks
    }

    private void calculateTotalCosts(ForeignTravel foreignTravel) {
        BigDecimal totalCostsToLcc = foreignTravel.getTravelCostsToLcc()
                .add(foreignTravel.getAccommodationCostsToLcc())
                .add(foreignTravel.getLocalTransportCostsToLcc())
                .add(foreignTravel.getFoodAndDrinkCostsToLcc());

        BigDecimal totalCostsToOther = foreignTravel.getTravelCostsToOther()
                .add(foreignTravel.getAccommodationCostsToOther())
                .add(foreignTravel.getLocalTransportCostsToOther())
   

In [ ]:
print(chat_history)

In [ ]:
#clear chat history if necessary
clear_chat_history(id)